<a href="https://colab.research.google.com/github/fatzzx/praticaBlockchain-Banco/blob/main/blockchain.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#------------------
#range
a = range(10)
for e in a:
  print(e, end=' ')

In [ ]:
#------------------
#len
b = len(a)
print(b)

In [ ]:
#------------------
l = [1, 2, 3, 4, 5, 6 ,7, 8, 9]
print(l[:5])
print(l[5:])
print(l[-1])

In [ ]:
#BLOCO 1
import hashlib

class Bloco:
    def __init__(self, index, dados, hash_anterior, dificuldade=4):
        self.index = index
        self.dados = dados
        self.hash_anterior = hash_anterior
        self.nonce = 0
        self.dificuldade = dificuldade
        self.hash = self.gerar_hash()

    def gerar_hash(self):
        hash_string = str(self.index) + str(self.dados) + str(self.hash_anterior) + str(self.nonce)
        encoded_string = hash_string.encode('utf-8')
        sha256_hash = hashlib.sha256()
        sha256_hash.update(encoded_string)
        hashed_output = sha256_hash.hexdigest()
        return hashed_output

    def minerar_bloco(self):
      self.nonce = 0
      self.hash = self.gerar_hash()
      while not self.validar():
          self.nonce += 1
          self.hash = self.gerar_hash()

    def validar(self):
        if self.hash[:self.dificuldade] != '0' * self.dificuldade:
            return False
        return True

    def __str__(self):
      return f"------------------\nIndex: {self.index}\nDados: {self.dados}\nHash Anterior: {self.hash_anterior}\nHash: {self.hash}"

In [ ]:
#BLOCO 2
class CadeiaBlocos:
  def __init__(self):
    self.cadeia = []
    self.cadeia.append(self.criar_bloco_genesis())
    self.minerar_ultimo_bloco()

  def criar_bloco_genesis(self):
    return Bloco(0, "Bloco Genesis", "0000")

  def add_bloco(self, dados):
      novo_bloco = Bloco(len(self.cadeia), dados, self.cadeia[-1].hash)
      if not novo_bloco.validar():
        novo_bloco.minerar_bloco()
      if novo_bloco.validar():
          self.cadeia.append(novo_bloco)
      else:
          print("Erro: bloco inválido não foi adicionado.")

  def atualiza_cadeia_a_partir(self, indice):
    for i in range(indice, len(self.cadeia)):
      self.cadeia[i].hash_anterior = self.cadeia[i-1].hash
      self.cadeia[i].hash = self.cadeia[i].gerar_hash()

  def minerar_ultimo_bloco(self):
    self.cadeia[-1].minerar_bloco()

  def minerar_todos_blocos(self):
    for i in range(0, len(self.cadeia)):
      self.cadeia[i].minerar_bloco()


  def mudar_dados(self, index, dados):
    if index >= 0 and index < len(self.cadeia):
      self.cadeia[index].dados = dados
      self.atualiza_cadeia_a_partir(index)
    else:
      print("Índice inválido")

  def verifica_cadeia(self):
    for b in self.cadeia:
      if b.validar():
        print(f"\033[92mBloco n {b.index} válido\033[0m")
        print(b)
      else:
        print(f"\033[91mBloco n {b.index} inválido\033[0m")
        print(b)

  def __str__(self):
    return '\n'.join(str(bloco) for bloco in self.cadeia)

In [ ]:
#BLOCO 3
blockchain = CadeiaBlocos()
blockchain.add_bloco("Transação 1")
blockchain.add_bloco("Transação 2")
blockchain.add_bloco("Transação 3")
blockchain.add_bloco("Transação 4")
blockchain.add_bloco("Transação 5")
blockchain.add_bloco("Transação 6")
blockchain.add_bloco("Transação 7")
blockchain.add_bloco("Transação 8")
blockchain.add_bloco("Transação 9")

In [ ]:
# - Altere manualmente os dados de um bloco na cadeia para corrompê-lo, com mudar_dados(numeroBloco, valorMudado), e depois verifique os estados da cadeia com o metodo verifica_cadeia()
#Escreva aqui





In [ ]:
#Aumente a dificuldade no bloco1, e tente rodar novamente o bloco 3

In [ ]:
class Pessoa:
    def __init__(self, nome):
        self.nome = nome
        self.cadeia_local = CadeiaBlocos()

    def adicionar_transacao(self, remetente, destinatario, valor):
        transacao = {"remetente": remetente.nome, "destinatario": destinatario.nome, "valor": valor}
        print(f"{remetente.nome} realizou uma transação para {destinatario.nome}, no valor de {valor}.")
        return transacao

    def atualiza_cadeia(self, cadeia):
      self.cadeia_local = cadeia

    def __str__(self):
        return f"{self.nome} - Blockchain Local: \n{self.blockchain_local}"

In [ ]:
class RedeBlockchain:
    def __init__(self, recompensa_mineracao=50):
        self.recompensa_mineracao = recompensa_mineracao
        self.transacoes_pendentes = []
        self.pessoas = []
        self.blockchain = CadeiaBlocos()

    def adicionar_transacao(self, remetente, destinatario, valor):
        transacao = {"remetente": remetente.nome, "destinatario": destinatario.nome, "valor": valor}
        self.transacoes_pendentes.append(transacao)
        print(f"Transação adicionada ao mempool: {transacao}")

    def adicionar_pessoa(self, pessoa):
        self.pessoas.append(pessoa)
        print(f"{pessoa.nome} foi adicionada à rede.")

    def atualiza_cadeia(self, bloco):
        print(f"Transmitindo o bloco {bloco.index} para a rede...")
        for pessoa in self.pessoas:
            pessoa.atualiza_cadeia(self.blockchain)

    def minerar_transacoes(self, minerador):
        if not self.transacoes_pendentes:
            print(f"Nenhuma transação para minerar")
            return

        dados_transacoes = self.transacoes_pendentes.copy()
        self.transacoes_pendentes.clear()

        dados_transacoes.append({"remetente": "Sistema", "destinatario": minerador.nome, "valor": self.recompensa_mineracao})

        novo_bloco = Bloco(len(self.blockchain.cadeia), dados_transacoes, self.blockchain.cadeia[-1].hash)
        novo_bloco.minerar_bloco()
        self.blockchain.cadeia.append(novo_bloco)

        print(f"{minerador.nome} minerou um bloco e recebeu {self.recompensa_mineracao} reais!")
        self.atualiza_cadeia(novo_bloco)

    def mostrar_transacoes_pendentes(self):
        print("Transações Pendentes (Mempool):")
        for transacao in self.transacoes_pendentes:
            print(transacao)

In [ ]:
fatzzx = Pessoa("fatzzx")
nika = Pessoa("Nika")
aragao = Pessoa("Aragão")
diogo = Pessoa("Diogo")

rede = RedeBlockchain()

rede.adicionar_pessoa(fatzzx)
rede.adicionar_pessoa(nika)
rede.adicionar_pessoa(aragao)
rede.adicionar_pessoa(diogo)

rede.adicionar_transacao(fatzzx, nika, 100)
rede.adicionar_transacao(nika, aragao, 50)
rede.adicionar_transacao(aragao, diogo, 200)
rede.adicionar_transacao(diogo, fatzzx, 150)

In [ ]:
rede.mostrar_transacoes_pendentes()

In [ ]:
soussa = Pessoa("Soussa")
rede.adicionar_pessoa(soussa)
rede.minerar_transacoes(soussa)

In [ ]:
rede.blockchain.verifica_cadeia()

In [ ]:
fatzzx.cadeia_local.verifica_cadeia()

In [ ]:
rede.mostrar_transacoes_pendentes()
rede.minerar_transacoes(soussa)